# Knowledge and Data: Practical Assignment 1 
## Manipulate Propositional Logic and Simple Knowledge Graph Logic

- YOUR NAME: Keimpe Rademaker

- YOUR VUNetID: kra225

*(If you do not provide your NAME and VUNetID we will not accept your submission.)*

### Learning objectives

At the end of this exercise you should be able to perform some simple manipulations of two different formal systems: 
1. You will be able to transform Propositional Logic statements from one syntactic form to semantically equivalent  alternative representation.
2. You will be able to evaluate a Propositional Logic statements given some validation of the propositional variables. 
3. You will be able to transform one Knowledge Graph into an equivalent one with a different Syntax 
4. You will be able to calculate simple entailment between two Knowledge Graph

### Practicalities

Follow this Notebook step-by-step. Some of the exercises are already 
fully filled in to give you some examples of how to do things. Others are
partially filled, and require you to fill in some gaps. And others, you 
will have to fully program yourself. 

Of course, you can do the exercises in any Programming Editor of your liking. 
But you do not have to. Feel free to simply write code in the Notebook. When 
everythink is filled in and works, save the Notebook and submit it 
as a Jupyter Notebook (i.e. with an ipynb extension). **Please use as name of the 
Notebook your studentID+Assignment1.ipynb**.  

We will not evaluate the programming style of your solutions. Yet we do look whether your solutions suggests an understanding, and whether they yield the correct output. 

Note that all notebooks will automatically be checked for plagiarism: while similar answers can be expected, it is not allowed to directly copy the solutions from fellow students or TAs, or from the examples discussed during the lectures. Similarly, sharing your solutions with your peers is not allowed.

## Exercises related to Propositional Logic 

We will use a very simple, though crude and ugly, representation for Propositional Logic: Propositions are strings, and formulas are lists, starting with an operator as first element in the list, and the subformulas as second and (if appropriate) third argument. (Note that this is slightly different than the definition in the lecture, where Propositions are lists with one argument). These are formulas in Prefix Notation.

In the following part, we will read several such formulas from a file (`PLknowlegebase.txt`).

For simplicity, we will only consider formulas with any of the three variables P, Q, and R, and binary operators ">>" (implies), "&" (and), "|" (or), and "~" (not). 

Here are some formulas as examples.

(~R & P) >> Q

~(((P | ~Q) >> R) >> (P & R)) 

~(P >> Q) | (R >> P) 

(P >> Q) | (Q >> ~P)


The first thing you will have to do is to open the file. The code below will read every line of the file into a list called *content*.

Do *not* forget to run the following cell, otherwise, the file will not be opened. import os 


In [1]:
import os 
from logic import *

In [2]:
fileDir = os.path.dirname(os.path.realpath('__file__'))
filename = os.path.join(fileDir, 'PLknowledgebase.txt')
print(filename)
with open(filename) as file:
    content = file.readlines()

print(content)

/Users/keimperademaker/Documents/GitHub/knowledge-data-vu/Assignments/Assignment_1/PLknowledgebase.txt
['(~R & P) >> Q\n', '~(((P | ~Q) >> R) >> (P & R)) \n', '~(P >> Q) | (R >> P) \n', '(P >> Q) | (Q >> ~P)']


In [3]:
P, Q, R = vars('P', 'Q', 'R')

for line in content:
    formula = eval(line.strip())
    print()
    print(formula)
    print(formula.is_tautology())
    print(type(formula))
    print(formula.children[0])


(¬R ^ P) => Q
False
<class 'logic.Implies'>
¬R ^ P

¬(((P v ¬Q) => R) => (P ^ R))
False
<class 'logic.Not'>
((P v ¬Q) => R) => (P ^ R)

¬(P => Q) v (R => P)
False
<class 'logic.Or'>
¬(P => Q)

(P => Q) v (Q => ¬P)
True
<class 'logic.Or'>
P => Q


The last procedure makes use of the inbuilt properties of the logic package. Let us do similar stuff ourselves. 

### Task 1: (10 Points) Complete the procedure *prefix* that transforms the formula into Prefix notation, i.e., the operator "sits" in front of the two arguments.

Use different symbols for the prefix operators: IMP for implication, AND for conjunction, OR for disjunction and NEG for negation. Look at the previous examples if you are unsure how to continue.

In [4]:
def prefix(s):
    if type(s) is Or:
        child1 = s.children[0]
        child2 = s.children[1]
        string = "OR(" + prefix(child1) + "," + prefix(child2) + ")";
        return string
    elif type(s) is And:
        child1 = s.children[0]
        child2 = s.children[1]
        string = "AND(" + prefix(child1) + "," + prefix(child2) + ")";
        return string
    elif type(s) is Not:
        child = s.children[0]
        string = "NEG(" + prefix(child) + ")";
        return string
    elif type(s) is Implies:
        child1 = s.children[0]
        child2 = s.children[1]
        string = "IMP(" + prefix(child1) + "," + prefix(child2) + ")";
        return string
    else:
        return str(s)

Execute the following line of code to check whether your procedure works correctly or not.

In [5]:
for line in content:
    formula = eval(line.strip())
    spacing = 30 - len(str(formula))
    print(str(formula) + " "*spacing + "== " + prefix(formula))

(¬R ^ P) => Q                 == IMP(AND(NEG(R),P),Q)
¬(((P v ¬Q) => R) => (P ^ R)) == NEG(IMP(IMP(OR(P,NEG(Q)),R),AND(P,R)))
¬(P => Q) v (R => P)          == OR(NEG(IMP(P,Q)),IMP(R,P))
(P => Q) v (Q => ¬P)          == OR(IMP(P,Q),IMP(Q,NEG(P)))


If you run your procedure on the input data, you should get something like the following output: 

    (¬R ^ P) => Q                 == IMP(AND(NEG(R),P),Q)
    ¬(((P v ¬Q) => R) => (P ^ R)) == NEG(IMP(IMP(OR(P,NEG(Q)),R),AND(P,R)))
    ¬(P => Q) v (R => P)          == OR(NEG(IMP(P,Q)),IMP(R,P))
    (P => Q) v (Q => ¬P)          == OR(IMP(P,Q),IMP(Q,NEG(P)))

### Task 2: (20 Points) Write a procedure *evaluate* that calculates the truth value of a formula. 

Given input formula _s_, and the three truth values for boolean variable P, Q, and R, write a procedure that tells us whether _s_ is true or false.

This is a procedural way to implement the "meaning" of the operators (which you can read from the Truth Tables)

In [6]:
def evaluate(s, p, q, r):

    ## Evaluating for disjunction
    if type(s) is Or:
        child1 = s.children[0]
        child2 = s.children[1]
        if (evaluate(child1,p,q,r) is True) or (evaluate(child2,p,q,r) is True):
            return True
        else:
            return False
        
    ## Evaluating for conjunction
    elif type(s) is And:
        child1 = s.children[0]
        child2 = s.children[1]
        if (evaluate(child1,p,q,r) is True) and (evaluate(child2,p,q,r) is True):
            return True
        else:
            return False
        
    ## Evaluating for negation   
    elif type(s) is Not:
        child = s.children[0]
        if evaluate(child,p,q,r) is False:
            return True
        else:
            return False
        
    ## Evaluating for implication
    elif type(s) is Implies:
        child1 = s.children[0]
        child2 = s.children[1]
        if (evaluate(child1,p,q,r) is False) and (evaluate(child2,p,q,r) is True):
            return True   
        elif (evaluate(child1,p,q,r) is True) and (evaluate(child2,p,q,r) is True):
            return True
        elif (evaluate(child1,p,q,r) is False) and (evaluate(child2,p,q,r) is False):
            return True
        else:
            return False

    ## Evaluating for variables
    elif type(s) is Variable:        
        if str(s) == 'P':
            return p
        elif str(s) == 'Q':
            return q
        elif str(s) == 'R':
            return r
        
    else:
        print("Something went wrong") 

Execute the following line of code to check whether your procedure works correctly or not.

In [7]:
for l in content:
    formula = l.strip()
    spacing = 30 - len(str(formula))
    print(str(formula) + " "*spacing + str(evaluate(s=eval(formula), p=True, q=True, r=False)))

(~R & P) >> Q                 True
~(((P | ~Q) >> R) >> (P & R)) False
~(P >> Q) | (R >> P)          True
(P >> Q) | (Q >> ~P)          True


If you run your program on the given input, you should get an answer like this:

    (~R & P) >> Q                 True
    ~(((P | ~Q) >> R) >> (P & R)) False
    ~(P >> Q) | (R >> P)          True
    (P >> Q) | (Q >> ~P)          True

### Task 3: (10 Points) Write a procedure *tautology* that calculates whether a given formula (with maximally three variables P,Q and R) is a tautology 
Hint: Use the *evaluate* function you just wrote. Do *not* use the 'is_tautology' method seen earlier.

In [8]:
def tautology(s):

    ## Evaluating for all possible truth values of P, Q, R
    for p in [True, False]:
        for q in [True, False]:
            for r in [True, False]:
                if evaluate(s,p,q,r) is False:
                    return False
    return True

    pass

Execute the following line of code to check whether your procedure works correctly or not.

In [9]:
print("Formula" + 23*" " + "Tautology")
for l in content:
    formula = l.strip()
    spacing = 30 - len(str(formula))
    print(str(formula) + " "*spacing + (repr(tautology(eval(formula)))))

Formula                       Tautology
(~R & P) >> Q                 False
~(((P | ~Q) >> R) >> (P & R)) False
~(P >> Q) | (R >> P)          False
(P >> Q) | (Q >> ~P)          True


When you evaluate your code on the provided input, you should get something like this:

    Formula                       Tautology
    (~R & P) >> Q                 False
    ~(((P | ~Q) >> R) >> (P & R)) False
    ~(P >> Q) | (R >> P)          False
    (P >> Q) | (Q >> ~P)          True

## Exercises related to Simple Knowledge Graph Logic 

We will use a very simple, though crude and ugly, representation for Simple Knowledge Graph Logic: Resources are strings, triples are tuples of resources, and knowledge graphs are lists of tuples. So, a triple ( s p o ) will be represented as a tuple `(s, p, o)`, and a Knowledge Graph with two triples (s1, p1, o1) and (s2, p2, o2) as a list containing two tuples: `[(s1,p1,o1), (s2,p2,o2)]`.

Every line in the file `knowledgegraph.txt` contains a knowledge graph (one list). 

Run the following code to load the Knowledge Graphs in knowledgegraph.txt

In [10]:
import os 
fileDir = os.path.dirname(os.path.realpath('__file__'))
filename = os.path.join(fileDir, 'knowledgegraph.txt')
with open(filename) as file:
    content = [[t for t in eval(g)] for g in file.readlines()]

Maybe you want to write a function to print this graph for convenience ... (no extra points, though) 

In [11]:
## Printing the content of the file
for g in content:
    print(g)

[('s', 'p', 'o'), ('s', 'p1', 'o2'), ('s2', 'p2', 'o')]
[('Netherlands', 'name', '"The Netherlands"'), ('Netherlands', 'has_capital', 'Amsterdam'), ('Amsterdam', 'type', 'Capital'), ('Amsterdam', 'has_population', '"921.402"')]
[('Netherlands', 'name', '"The Netherlands"'), ('Netherlands', 'has_capital', 'Amsterdam'), ('Amsterdam', 'type', 'Capital'), ('Amsterdam', 'has_population', '"921.402"'), ('Capital', 'type', 'City'), ('Netherlands', 'neighbours', 'Belgium'), ('Netherlands', 'type', 'Country')]


The next two exercises are about syntactic transformation of Knowledge Graphs. The two target formats are simplified versions of syntaxes for the language RDF that we will introduce later. 

### Task 4a: (10 Points) Write a procedure to transform the Knowledge Graphs into N-triple syntax. 
N-triple is a simple (the most simple?) representation format for representing Knowledge Graphs. 

A triple (s,p,o) is simply written as as follows:

     <s> <p> <o> . 
     
As one string, with blanks between s,p and o, with < and > around the variables, and a dot after the triple. 

First, write a function that transforms a triple ('s','p','o') into a single string "\<s> \<p> \<o> ."

In [12]:
def as_ntriple(triple):
    ## Converting the triple to N-Triple format
    a, b, c = triple
    return f"<{a}> <{b}> <{c}> ."

In [13]:
def as_ntriples(graph):
    ## ... and then loops through all triples in the graph (hint: you can reuse the function above)
    return ([as_ntriple(triple) for triple in graph])
    pass

Execute the following line of code to check whether your procedure works correctly or not.

In [14]:
for graph in content:
     print(as_ntriples(graph))

['<s> <p> <o> .', '<s> <p1> <o2> .', '<s2> <p2> <o> .']
['<Netherlands> <name> <"The Netherlands"> .', '<Netherlands> <has_capital> <Amsterdam> .', '<Amsterdam> <type> <Capital> .', '<Amsterdam> <has_population> <"921.402"> .']
['<Netherlands> <name> <"The Netherlands"> .', '<Netherlands> <has_capital> <Amsterdam> .', '<Amsterdam> <type> <Capital> .', '<Amsterdam> <has_population> <"921.402"> .', '<Capital> <type> <City> .', '<Netherlands> <neighbours> <Belgium> .', '<Netherlands> <type> <Country> .']


Your answer should look similar to this: <br />
['\<s> \<p> \<o> .', '\<s> \<p1> \<o2> .', '\<s2> \<p2> \<o> .']<br/>
['\<Netherlands> \<name> \<"Netherlands"> .', '\<Netherlands> \<has_capital> \<Amsterdam> .', '\<Amsterdam> \<type> \<Capital> .']<br/>
['\<Netherlands> \<name> \<"Netherlands"> .', '\<Netherlands> \<has_capital> \<Amsterdam> .', '\<Amsterdam> \<type> \<Capital> .', '\<Capital> \<type> \<City> .', '\<Netherlands> \<neighbours> \<Belgium> .', '\<Netherlands> \<type> \<Country> .']

### Task 4b (10 Points) Adjust your procedure to cope with literal objects

In reality, the N-triples format specifies that only objects must be embedded between angled brackets; literal values, such as strings and numbers, are to be represented as string values, for example "\<s> \<p> \"12\" .". Since knowledge graphs are construced with an object-centric language (RDF), we only have to consider the last element of a triple.

Redefine the function _as\_triple_ to make it output valid N-triples.

In [15]:
def as_ntriple(triple):

    ## Converting the triple to N-Triple format, removing angle brackets if the object is a string, and if the object is enclosed in quotation marks
    a, b, c = triple
    if isinstance(c, str) and ('"' in c or "'" in c):
        return f"<{a}> <{b}> {c} ."
    else:
        return f"<{a}> <{b}> <{c}> ."
    pass

Execute the following line of code to check whether your procedure works correctly or not.

In [16]:
for graph in content:
     print(as_ntriples(graph))

['<s> <p> <o> .', '<s> <p1> <o2> .', '<s2> <p2> <o> .']
['<Netherlands> <name> "The Netherlands" .', '<Netherlands> <has_capital> <Amsterdam> .', '<Amsterdam> <type> <Capital> .', '<Amsterdam> <has_population> "921.402" .']
['<Netherlands> <name> "The Netherlands" .', '<Netherlands> <has_capital> <Amsterdam> .', '<Amsterdam> <type> <Capital> .', '<Amsterdam> <has_population> "921.402" .', '<Capital> <type> <City> .', '<Netherlands> <neighbours> <Belgium> .', '<Netherlands> <type> <Country> .']


Your answer should look similar to this: <br />
['\<s> \<p> \<o> .', '\<s> \<p1> \<o2> .', '\<s2> \<p2> \<o> .']<br/>
['\<Netherlands> \<name> "Netherlands" .', '\<Netherlands> \<has_capital> \<Amsterdam> .', '\<Amsterdam> \<type> \<Capital> .',  '\<Amsterdam> \<has_population> "921.402" .']<br/>
['\<Netherlands> \<name> "Netherlands" .', '\<Netherlands> \<has_capital> \<Amsterdam> .', '\<Amsterdam> \<type> \<Capital> .', '\<Capital> \<type> \<City> .', '\<Netherlands> \<neighbours> \<Belgium> .', '\<Netherlands> \<type> \<Country> .',  '\<Amsterdam> \<has_population> "921.402" .']

### Task 5: (20 Points) Write a procedure to evaluate whether a Triple (s p o) is entailed by your Knowledge Graph

Remember that we can calculate whether a triple is entailed by a Knowledge Graph (w.r.t Simple Knowledge Graph Logic) by simply checking whether the triple is part of the knowledge graph.  

The triples, on which you can test your implementations are located in the file: *entailedTriples.txt*. You need to open the file and check for all triples whether it is entailed by the knowledge graphs in *knowledgegraph.txt* (which is already opened). Hint: you can use *eval* to convert a raw string to a list.

You might want to add auxiliary functions in extra cells if necessary.

In [17]:
def tripleEntailedBy(entailedTriples):
    
    ## Looping through all knowledge graphs in the content, as well as all entailed triples
    for knowledgeGraph in content:
        for entailedTriple in entailedTriples:

            ## Checking if the entailed triple is in the knowledge graph
            if entailedTriple in knowledgeGraph:
                print(str(knowledgeGraph) + " entails " + str(entailedTriple))
            else:
                print(str(knowledgeGraph) + " does NOT entail " + str(entailedTriple))
    pass

Provide some code to check that your function works correctly.  

In [18]:
filename = os.path.join(fileDir, 'entailedTriples.txt')
with open(filename) as file:
    triples = [eval(t) for t in file.readlines()]
    
# your code here
tripleEntailedBy(triples)

[('s', 'p', 'o'), ('s', 'p1', 'o2'), ('s2', 'p2', 'o')] entails ('s2', 'p2', 'o')
[('s', 'p', 'o'), ('s', 'p1', 'o2'), ('s2', 'p2', 'o')] does NOT entail ('Netherlands', 'name', '"The Netherlands"')
[('Netherlands', 'name', '"The Netherlands"'), ('Netherlands', 'has_capital', 'Amsterdam'), ('Amsterdam', 'type', 'Capital'), ('Amsterdam', 'has_population', '"921.402"')] does NOT entail ('s2', 'p2', 'o')
[('Netherlands', 'name', '"The Netherlands"'), ('Netherlands', 'has_capital', 'Amsterdam'), ('Amsterdam', 'type', 'Capital'), ('Amsterdam', 'has_population', '"921.402"')] entails ('Netherlands', 'name', '"The Netherlands"')
[('Netherlands', 'name', '"The Netherlands"'), ('Netherlands', 'has_capital', 'Amsterdam'), ('Amsterdam', 'type', 'Capital'), ('Amsterdam', 'has_population', '"921.402"'), ('Capital', 'type', 'City'), ('Netherlands', 'neighbours', 'Belgium'), ('Netherlands', 'type', 'Country')] does NOT entail ('s2', 'p2', 'o')
[('Netherlands', 'name', '"The Netherlands"'), ('Netherla

The expected output is a list of relations between graphs and triples<br />
[('s','p','o'),('s','p1','o2'),('s2','p2','o')] entails ('s2','p2','o')<br />
[('s','p','o'),('s','p1','o2'),('s2','p2','o')] does NOT entail ('Netherlands','name','"The Netherlands"')<br />
and so on for all graphs and triples. 

### Task 6: (20 Points) Write a procedure to evaluate whether one Knowledge Graph is entailed by another Knowledge Graph

Remember that we can calculate whether a triple is entailed by another graph (w.r.t Simple Knowledge Graph Logic) by simply checking whether the former is a subgraph of the latter. The same holds for graphs.

Write a procedure to check for all of graphs in `entailedGraphs.txt` whether they are entailed by those in `knowledgegraph.txt` (which is already opened).

You might want to add auxiliary functions in extra cells if necessary.

In [19]:
def graphEntailedBy(entailedGraphs):

    ## Converting the triples to N-Triples format, replacing instances of 'a' with 'type'
    def convertTriple(triple):
        return (triple[0], 'type' if triple[1] == 'a' else triple[1], triple[2])
    
    ## Looping through all knowledge graphs in the content, as well as all entailed graphs
    for knowledgeGraph in content:
        for entailedGraph in entailedGraphs:
            
            ## Creates transformedGraph by converting all triples in entailedGraph using the convertTriple() function above
            transformedGraph = [convertTriple(triple) for triple in entailedGraph]

            ## Checks if all triples in transformedGraph are in the knowledge graph
            if all(triple in knowledgeGraph for triple in transformedGraph):
                print(str(knowledgeGraph) + " entails " + str(entailedGraph))
            else:
                print(str(knowledgeGraph) + " does NOT entail " + str(entailedGraph))
    pass

Provide some code to check that your function works correctly.  

In [20]:
filename = os.path.join(fileDir, 'entailedGraphs.txt')
with open(filename) as file:
    graphs = [[t for t in eval(g)] for g in file.readlines()]

# your code here
graphEntailedBy(graphs)

[('s', 'p', 'o'), ('s', 'p1', 'o2'), ('s2', 'p2', 'o')] entails [('s2', 'p2', 'o'), ('s2', 'p2', 'o')]
[('s', 'p', 'o'), ('s', 'p1', 'o2'), ('s2', 'p2', 'o')] entails [('s', 'p', 'o'), ('s2', 'p2', 'o')]
[('s', 'p', 'o'), ('s', 'p1', 'o2'), ('s2', 'p2', 'o')] does NOT entail [('Netherlands', 'name', '"The Netherlands"'), ('Netherlands', 'has_capital', 'Amsterdam'), ('Capital', 'a', 'City'), ('Netherlands', 'neighbours', 'Belgium')]
[('s', 'p', 'o'), ('s', 'p1', 'o2'), ('s2', 'p2', 'o')] does NOT entail [('Netherlands', 'has_capital', 'Amsterdam'), ('Capital', 'a', 'Country'), ('Netherlands', 'neighbours', 'Belgium')]
[('Netherlands', 'name', '"The Netherlands"'), ('Netherlands', 'has_capital', 'Amsterdam'), ('Amsterdam', 'type', 'Capital'), ('Amsterdam', 'has_population', '"921.402"')] does NOT entail [('s2', 'p2', 'o'), ('s2', 'p2', 'o')]
[('Netherlands', 'name', '"The Netherlands"'), ('Netherlands', 'has_capital', 'Amsterdam'), ('Amsterdam', 'type', 'Capital'), ('Amsterdam', 'has_pop

The expected output is a list of relations between graphs and graphs<br />
[('s', 'p', 'o'), ('s', 'p1', 'o2'), ('s2', 'p2', 'o')] entails [('s2', 'p2', 'o'), ('s2', 'p2', 'o')]<br />
[('s', 'p', 'o'), ('s', 'p1', 'o2'), ('s2', 'p2', 'o')] entails [('s', 'p', 'o'), ('s2', 'p2', 'o')]<br />
[('s', 'p', 'o'), ('s', 'p1', 'o2'), ('s2', 'p2', 'o')] does NOT entail [('Netherlands', 'name', '"The Netherlands"'), ('Netherlands', 'has_capital', 'Amsterdam'), ('Capital', 'a', 'City'), ('Netherlands', 'neighbours', 'Belgium')]
<br />
and so on for all graphs and triples. 